In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Current Status,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,KL,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(8100, 26)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
State code                                   0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Type of transmission                         0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(8100, 19)
Patient Number 	 343
State Patient Number 	 6640
Date Announced 	 482
Age Bracket 	 6940
Gender 	 6199
Detected City 	 7104
Detected District 	 2116
Detected State 	 482
State code 	 482
Current Status 	 483
Notes 	 1674
Contracted from which Patient (Suspected) 	 6796
Nationality 	 6915
Type of transmission 	 5276
Status Change Date 	 592
Source_1 	 688
Source_2 	 5917
Source_3 	 7798
Backup Notes 	 7740


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(8100, 19)


Patient Number                                343
State Patient Number                         6640
Date Announced                                482
Age Bracket                                  6940
Gender                                       6199
Detected City                                7104
Detected District                            2116
Detected State                                482
State code                                    482
Current Status                                483
Notes                                        1674
Contracted from which Patient (Suspected)    6796
Nationality                                  6915
Type of transmission                         5276
Status Change Date                            592
Source_1                                      688
Source_2                                     5917
Source_3                                     7798
Backup Notes                                 7740
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(7618, 19)


Patient Number                                  0
State Patient Number                         6158
Date Announced                                  0
Age Bracket                                  6458
Gender                                       5717
Detected City                                6622
Detected District                            1634
Detected State                                  0
State code                                      0
Current Status                                  1
Notes                                        1192
Contracted from which Patient (Suspected)    6314
Nationality                                  6433
Type of transmission                         4794
Status Change Date                            110
Source_1                                      206
Source_2                                     5435
Source_3                                     7316
Backup Notes                                 7258
dtype: int64

In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')

In [15]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
567,568,NaN,24/03/2020,36,M,Rajkot,Rajkot,Gujarat,GJ,Hospitalized,Travelled from Saudi Arabia,NaN,India,Imported,NaN,NaN,NaN,NaN,NaN
6578,6579,NaN,09/04/2020,NaN,NaN,NaN,Patan,Gujarat,GJ,Hospitalized,Details awaited,NaN,NaN,NaN,09/04/2020,https://twitter.com/ANI/status/124825922680940...,NaN,NaN,NaN
5646,5647,NaN,08/04/2020,NaN,NaN,NaN,Rampur,Uttar Pradesh,UP,Hospitalized,NaN,NaN,NaN,NaN,08/04/2020,https://t.me/indiacovid/1843,https://twitter.com/TOIHyderabad/status/124789...,NaN,NaN
3734,3735,NaN,05/04/2020,NaN,NaN,NaN,Mumbai,Maharashtra,MH,Hospitalized,NaN,NaN,NaN,NaN,05/04/2020,https://twitter.com/ANI/status/124673842837022...,NaN,NaN,NaN
3844,3845,TN-P494,05/04/2020,NaN,M,NaN,Chennai,Tamil Nadu,TN,Hospitalized,Travelled to Delhi,E0,NaN,Local,05/04/2020,State Bulletin: 05 Apr Evening,NaN,NaN,NaN


In [16]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'state_code', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'type_of_transmission',
       'status_change_date', 'source_1', 'source_2', 'source_3',
       'backup_notes', 'p_id'],
      dtype='object')

In [17]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [18]:
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
6911,6912,NaN,10/04/2020,54,NaN,Thiruvandarkoil,Puducherry,Puducherry,PY,Hospitalized,Delhi Conference,E0,India,Local,10/04/2020,https://twitter.com/PTI_News/status/1248555682...,https://collectorate.py.gov.in/Corona%20COVID-...,NaN,NaN
5192,5193,NaN,07/04/2020,NaN,NaN,NaN,Jogulamba Gadwal,Telangana,TG,Hospitalized,Details awaited,NaN,NaN,NaN,07/04/2020,https://twitter.com/IPRTelangana/status/124755...,NaN,NaN,NaN
3581,3582,NaN,04/04/2020,NaN,NaN,NaN,Pune,Maharashtra,MH,Hospitalized,NaN,NaN,NaN,NaN,NaN,https://twitter.com/rajeshtope11/status/124648...,NaN,NaN,NaN
3393,3394,TN-P426,04/04/2020,NaN,M,NaN,Madurai,Tamil Nadu,TN,Hospitalized,Travelled to Delhi,E0,NaN,Local,04/04/2020,https://twitter.com/ANI/status/124641832005770...,NaN,NaN,NaN
3871,3872,TN-P521,05/04/2020,NaN,M,NaN,Coimbatore,Tamil Nadu,TN,Hospitalized,Travelled to Delhi,E0,NaN,Local,05/04/2020,State Bulletin: 05 Apr Evening,NaN,NaN,NaN


In [19]:
p_df.to_csv('patients_data.csv', index=False)